In [2]:
import pandas as pd
import numpy as np

#### Cleaning the Censo dataset

This dataset contains information on the average price of rent in different neighborhoods of Barcelona

In [3]:
# Estimacion del precio mensual de locales commerciales por barrio en años 2011, 2010, 2009, 2008
df1 = pd.read_csv('../Datos CSV/locveevolucio.csv')
relabel = {'BARRIS': 'NOM', 'DOSZEROONZE': '2011', 'DOSZERODEU': '2010', 'DOSZEROZERONOU': '2009', 'DOSZEROZEROVUIT': '2008'}
df1.rename(columns=relabel, inplace=True)
df1.drop(columns='DTE', inplace=True)

# Function to clean the data
def replace_characters(df, columns):
    for column in columns:
        df[column] = df[column].str.replace('[\.,]', '')
        df[column] = df[column].str.strip()
    return df

def remove_numbers(df, columns):
    for column in columns:
        df[column] = df[column].str.replace('\d', '')
        df[column] = df[column].str.strip()
    return df

import re
import emoji

def remove_numbers_robust(df, columns):
    for column in columns:
        # Remove numbers
        df[column] = df[column].str.replace('\d', '')
        
        # Remove links
        df[column] = df[column].apply(lambda text: re.sub(r'http\S+', '', text))
        
        # Remove emojis
        df[column] = df[column].apply(lambda text: emoji.get_emoji_regexp().sub(r'', text))
        
        # Strip leading and trailing whitespace
        df[column] = df[column].str.strip()
    
    return df


In [4]:
# Cleaning the data, applying the functions
replace_characters(df1, ['NOM', '2011', '2010', '2009', '2008'])
remove_numbers(df1, ['NOM'])

df1 = df1[~(df1 == '--').any(axis=1)]
# Correct data types
df1[['2011', '2010', '2009', '2008']] = df1[['2011', '2010', '2009', '2008']].astype(int)

df1['price level'] = df1['2011'] - df1['2011'].mean()

/var/folders/hp/ytf1kqk92hd11vlmtk9zz38r0000gn/T/ipykernel_89426/2070660899.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace('[\.,]', '')
/var/folders/hp/ytf1kqk92hd11vlmtk9zz38r0000gn/T/ipykernel_89426/2070660899.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column] = df[column].str.replace('\d', '')


In [5]:
# df1.to_csv('../Datos CSV/precios_clean.csv', index=False)

### Censo Comercial Dataset

In [6]:
# Load dataset
df2 = pd.read_csv('../Datos CSV/2019_censcomercialbcn_detall.csv')

# Apply column selection, rename columns and replace values
# direccion = 'Codi_Via', 'Nom_Via', 'Planta', 'Porta','Lletra_Inicial'
censo_cols = ['ID_Bcn_2019','Nom_Principal_Activitat','Nom_Activitat','Nom_Sector_Activitat','Nom_Grup_Activitat','Nom_Local','Latitud','Longitud','Nom_Districte', 'Nom_Barri']
censo_clean = pd.DataFrame(df2[censo_cols])
censo_clean.rename(columns={'Nom_Principal_Activitat':'Actividad Economica','Nom_Activitat':'Nombre de Actividad', 'Nom_Barri': 'Nombre del Barrio'}, inplace=True)
censo_clean['Actividad Economica'].replace({'Actiu': 'For Profit', 'Sense activitat Econòmica': 'Non Profit'}, inplace=True)

/Users/joaquinortega/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: DtypeWarning: Columns (35,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
first_lambda = ['Sense informació','altres','Altres ( per exemple VENDING)', 'Altres']
# Cambiar con la tercera columna - Nom_Grup_Activitat = First Lambda
censo_clean['Nombre de Actividad'] = censo_clean.apply(lambda row: row['Nom_Grup_Activitat'] if row['Nombre de Actividad'] in first_lambda else row['Nombre de Actividad'], axis=1)
# Cambiar con la segunda columna - Nom_Sector_Activitat
censo_clean['Nombre de Actividad'] = censo_clean.apply(lambda row: row['Nom_Sector_Activitat'] if row['Nombre de Actividad'] in first_lambda else row['Nombre de Actividad'], axis=1)

In [8]:
# pd.set_option('display.max_rows', None)
# censo_clean['Nombre de Actividad'].value_counts()

In [9]:
drop_cols = ['Actividad Economica','ID_Bcn_2019', 'Nom_Sector_Activitat', 'Nom_Grup_Activitat', 'Nom_Districte']
censo_clean.drop(columns=drop_cols, inplace=True)

In [10]:
trad_df = pd.DataFrame(censo_clean['Nombre de Actividad'].unique())
trad_df['name_english'] = ''
trad_df.rename(columns={0: 'name_spanish'}, inplace=True)
trad_df

,name_spanish,name_english
0,Serveis a les empreses i oficines,
1,Activitats emmagatzematge,
2,Administració,
3,Activitats de transport,
4,Vehicles,
...,...,...
80,Locutoris,
81,Maquinària,
82,Esports,
83,Òptiques i fotografia,


In [11]:
# from translate import Translator
# from tqdm import tqdm

# def translate_catalan_to_english(text):
#     translator = Translator(from_lang='ca', to_lang='en')
#     translated_text = translator.translate(text)
#     return translated_text

# total_rows = len(a_traducir)
# tqdm.pandas(total=total_rows)  # Initialize tqdm with total rows

NameError: name 'a_traducir' is not defined

In [12]:
# trad_df['name_english'] = trad_df['name_spanish'].progress_apply(translate_catalan_to_english)

AttributeError: 'Series' object has no attribute 'progress_apply'

In [13]:
trad_complete = pd.DataFrame(censo_clean['Nombre de Actividad'])
trad_complete['all_names_english'] = ''
trad_complete.head(3)

,Nombre de Actividad,all_names_english
0,Serveis a les empreses i oficines,
1,Activitats emmagatzematge,
2,Serveis a les empreses i oficines,


In [14]:
# Assuming `trad_complete` and `trad_df` are DataFrames with the respective columns

trad_complete['all_names_english'] = ''

total_rows = len(trad_complete)
with tqdm(total=total_rows, desc='Progress') as pbar:
    for i, activity_name in trad_complete['Nombre de Actividad'].iteritems():
        matching_index = trad_df[trad_df['name_spanish'] == activity_name].index

        if len(matching_index) > 0:
            matching_english = trad_df.loc[matching_index, 'name_english'].values[0]
            trad_complete.loc[i, 'all_names_english'] = matching_english

        pbar.update(1)

print(trad_complete)

Progress:   2%|▏         | 1365/80554 [00:02<02:16, 579.09it/s]


KeyboardInterrupt: 

In [345]:
english_censo_df= trad_complete.merge(censo_clean, left_index=True, right_index=True)

In [351]:
# Code to check translation went well for the big_df
# counter = 0
# for i in range(1, len(english_censo_df)):
#     if english_censo_df['Nombre de Actividad_y'][i] == english_censo_df['Nombre de Actividad_x'][i]:
#         counter += 1
#     pass

In [360]:
# english_censo_df.drop(columns=['Nombre de Actividad_y'], inplace=True)
english_censo_df.rename(columns={'Nombre de Actividad_x': 'Nombre de Actividad','all_names_english':'Name of Activity'}, inplace=True)

In [362]:
english_censo_df.drop(columns=['Nombre de Actividad'], inplace=True)
english_censo_df

,Name of Activity,Nom_Local,Latitud,Longitud,Nombre del Barrio
0,Services to companies and offices,SORIGUE,41.346101,2.130166,la Marina del Prat Vermell
1,Storage activities,CEJIDOS SIVILA S.A,41.345939,2.129560,la Marina del Prat Vermell
2,Services to companies and offices,QUALITY ESPRESO,41.345591,2.128543,la Marina del Prat Vermell
3,Administration,CLD,41.346262,2.130599,la Marina del Prat Vermell
4,Transport activities,"CATALANA DEL BUTANO,S.A",41.346514,2.131271,la Marina del Prat Vermell
...,...,...,...,...,...
80549,Other sports facilities,COSTURART,41.422618,2.177690,el Guinardó
80550,Transport activities,ABRILBIKE,41.399403,2.178403,el Fort Pienc
80551,Other sports facilities,FUTBOL SALA VALLDAURA,41.441363,2.161877,Horta
80552,Religious facilities,ESGLESIA ORTODOXA PATRIARCAT DE SERBIA,41.387255,2.158082,l'Antiga Esquerra de l'Eixample


In [364]:
english_censo_df.to_csv('../Datos CSV/english_censo_clean.csv', index=False)

In [22]:
# censo_clean.to_json('../JSON listos/censo_clean.json', orient='records')  

### CSV Centro Comercial

In [130]:
# Load the CSV - Centro Comercial
centros_comerciales = pd.read_csv('../Datos CSV/opendatagrans-centres-comercials.csv', encoding='utf-16')
# Add new column for categorization
centros_comerciales['Tipo_Centro_Comercio'] = 'Centro Comercial'

### Galerias Comerciales

In [132]:
# Load the CSV - Galerias Comerciales
galerias = pd.read_csv('../Datos CSV/opendata_galeries-comercials.csv', encoding='utf-16')
# Add new column for categorization - Galerias Comerciales
galerias['Tipo_Centro_Comercio'] = 'Galeria Comercial'

### Mercats Fires al Carrer

In [134]:
# Add new column for categorization - Galerias Comerciales
mercats_fires = pd.read_csv('../Datos CSV/opendata_mercats-fires-al-carrer.csv', encoding='utf-16')
mercats_fires['Tipo_Centro_Comercio'] = 'Feria'

### Mercados Municipales

In [136]:
mercats_municipals = pd.read_csv('../Datos CSV/opendata_mercats-municipals.csv', encoding='utf-16')
mercats_municipals['Tipo_Centro_Comercio'] = 'Mercado Municipal'

### Concatenation of Centros de Comercio

In [138]:
comercio_bcn = pd.concat([centros_comerciales, galerias, mercats_fires, mercats_municipals])
comercio_cols = ['register_id','name','addresses_road_name','addresses_start_street_number','addresses_district_name','addresses_zip_code','geo_epgs_4326_x','geo_epgs_4326_y','Tipo_Centro_Comercio']
filtered_comercio = pd.DataFrame(comercio_bcn[comercio_cols])
filtered_comercio